In [ ]:
rm(list = ls())
setwd("/home/creambbq/facu/RIF/TP2")
library(tidyverse); library(readxl); library(ggthemes)

returns <- read_xls(path = "retorn1.xls",
                    sheet = "Sheet1",
                    col_names = as.character(seq(1, 22)))
returns <- returns / 100

In [3]:
get_alpha <- function(ret, rf, Wo, fun_type, param = 0) {
    if (fun_type == "CES") {
        gamma = param
        RA_inv_wo <- (1 + rf) / gamma
    } else if (fun_type == "exponencial") {
        delta <- param
        RA_inv_wo <- 1 / (delta * Wo)
    } else if (fun_type == "cuadratica") {
        RA_inv_wo <- (5 - 2 * Wo * (1 + rf)) / (2 * Wo)
    } else {
        stop("Tipo de función invalida")
    }
    exc_ret <- as.matrix(ret - rf)
    exc_ret_medio <- as.matrix(colMeans(ret))
    R_media <- cov(exc_ret) + exc_ret_medio %*% t(exc_ret_medio)
    alpha <- RA_inv_wo * (solve(R_media) %*%  exc_ret_medio)
    return(alpha)
}

#### Inciso a)
Utilizando una función de utilidad CES, es decir, 

$$ u(z) = \dfrac{z^{1-\gamma}}{1-\gamma} $$ 

Se llega a que el portafolio del agente se compone de una operación en corto en el activo libre de riesgo de magnitud aproximada 2.88 para luego invertir en los distintos activos riesgosos con las siguiente proporciones: 

In [4]:
alpha1 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 4,
                    fun_type = "CES",
                    param = 5)
a <- alpha1 * 4
(t(a) / sum(a))

1,2,3,4,5,6,7,8,9,10,⋯,13,14,15,16,17,18,19,20,21,22
-0.3294931,-0.2714308,-0.3762617,-0.1077539,-0.02407146,-0.08531864,-0.05188291,0.633168,-0.3273896,0.7971688,⋯,1.710047,-0.1841814,-0.2580663,-0.222127,-0.002964922,-0.5568716,-0.4872657,0.3970215,0.511915,0.04119072


#### Inciso b)

Los resultados obtenidos en el inciso anterior surgen de una serie de expansiones de Taylor y arrojan un resultado, para este caso, aproximado. El parámetro $\alpha$, del que se deriva el portafolio y sus proporciones, no depende de la riqueza inicial del agente en este caso dado que,

$$ - \dfrac{u^{''}(W_0(1+r_f))W_0}{u^{'}(W_0(1+r_f))} = \dfrac{\gamma}{1+r_f} $$

$$ \implies \alpha = \dfrac{1+r_f}{\gamma} [E(R)]^{-1} E(r - r_f) $$

Teniendo esto en cuenta, el portafolio se va a mantener inalterado ante el cambio en $W_0$ propuesto por el ejercicio. 


In [5]:
alpha2 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 0,
                    fun_type = "CES",
                    param = 5)
                    
alpha3 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 8,
                    fun_type = "CES",
                    param = 5)

t(data.frame("Wo_0" = alpha2, "Wo_8" = alpha3))


,1,2,3,4,5,6,7,8,9,10,⋯,13,14,15,16,17,18,19,20,21,22
Wo_0,-0.5668607,-0.4669702,-0.6473214,-0.18538,-0.0414126,-0.1467824,-0.08925948,1.089304,-0.5632417,1.371451,⋯,2.941969,-0.3168662,-0.4439778,-0.3821477,-0.005100859,-0.9580431,-0.838293,0.6830367,0.8806997,0.07086461
Wo_8,-0.5668607,-0.4669702,-0.6473214,-0.18538,-0.0414126,-0.1467824,-0.08925948,1.089304,-0.5632417,1.371451,⋯,2.941969,-0.3168662,-0.4439778,-0.3821477,-0.005100859,-0.9580431,-0.838293,0.6830367,0.8806997,0.07086461


ccc

In [10]:
alpha5 <- get_alpha(ret = returns,
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 1,
                    fun_type = "exponencial",
                    param = 3)
                    
alpha6 <- get_alpha(ret = returns,
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 2,
                    fun_type = "exponencial",
                    param = 3)
            
alpha7 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 3,
                    fun_type = "exponencial",
                    param = 3)

t(data.frame("Wo_1" = alpha5 * 1, "Wo_2" = alpha6 * 2, "Wo_3" = alpha7 * 3))
data.frame("Wo_1" = alpha5 / sum(alpha5),
           "Wo_2" = alpha6 / sum(alpha6),
           "Wo_3" = alpha7 / sum(alpha7))

,1,2,3,4,5,6,7,8,9,10,⋯,13,14,15,16,17,18,19,20,21,22
Wo_1,-0.9446787,-0.7782103,-1.078767,-0.3089376,-0.06901449,-0.2446142,-0.1487518,1.815335,-0.9386478,2.285536,⋯,4.902819,-0.5280605,-0.7398933,-0.6368528,-0.00850063,-1.596588,-1.397023,1.138287,1.467694,0.1180966
Wo_2,-0.9446787,-0.7782103,-1.078767,-0.3089376,-0.06901449,-0.2446142,-0.1487518,1.815335,-0.9386478,2.285536,⋯,4.902819,-0.5280605,-0.7398933,-0.6368528,-0.00850063,-1.596588,-1.397023,1.138287,1.467694,0.1180966
Wo_3,-0.9446787,-0.7782103,-1.078767,-0.3089376,-0.06901449,-0.2446142,-0.1487518,1.815335,-0.9386478,2.285536,⋯,4.902819,-0.5280605,-0.7398933,-0.6368528,-0.00850063,-1.596588,-1.397023,1.138287,1.467694,0.1180966


,Wo_1,Wo_2,Wo_3
,<dbl>,<dbl>,<dbl>
1,-0.329493108,-0.329493108,-0.329493108
2,-0.271430843,-0.271430843,-0.271430843
3,-0.376261658,-0.376261658,-0.376261658
4,-0.107753895,-0.107753895,-0.107753895
5,-0.024071463,-0.024071463,-0.024071463
6,-0.085318644,-0.085318644,-0.085318644
7,-0.051882914,-0.051882914,-0.051882914
8,0.633168004,0.633168004,0.633168004
9,-0.327389586,-0.327389586,-0.327389586


dddd

In [11]:
alpha8 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 2,
                    fun_type = "exponencial",
                    param = 5)
                    
t(alpha8)

1,2,3,4,5,6,7,8,9,10,⋯,13,14,15,16,17,18,19,20,21,22
-0.2834036,-0.2334631,-0.3236302,-0.09268128,-0.02070435,-0.07338427,-0.04462553,0.5446005,-0.2815943,0.6856608,⋯,1.470846,-0.1584181,-0.221968,-0.1910558,-0.002550189,-0.4789764,-0.419107,0.3414862,0.4403083,0.03542897


eeee

In [13]:
alpha9 <- get_alpha(ret = returns,
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 1,
                    fun_type = "cuadratica",
                    param = 3)
                    
alpha10 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 2,
                    fun_type = "cuadratica",
                    param = 3)
            
alpha11 <- get_alpha(ret = returns, 
                    rf = (1 + 0.035)^(1 / 365) - 1,
                    Wo = 3,
                    fun_type = "cuadratica",
                    param = 3)

t(data.frame("Wo_1" = alpha9, "Wo_2" = alpha10 * 2, "Wo_3" = alpha11 * 3))
data.frame("Wo_1" = alpha9 / sum(alpha9),
           "Wo_2" = alpha10 / sum(alpha10),
           "Wo_3" = alpha11 / sum(alpha11))

,1,2,3,4,5,6,7,8,9,10,⋯,13,14,15,16,17,18,19,20,21,22
Wo_1,-4.250787,-3.501726,-4.854148,-1.3901318,-0.3105457,-1.1006949,-0.6693409,8.168494,-4.223650,10.284266,⋯,22.061300,-2.3761229,-3.329311,-2.8656576,-0.03825043,-7.184195,-6.286210,5.121971,6.604210,0.5314011
Wo_2,-1.416484,-1.166875,-1.617541,-0.4632317,-0.1034827,-0.3667830,-0.2230435,2.721976,-1.407441,3.427012,⋯,7.351456,-0.7917921,-1.109422,-0.9549191,-0.01274614,-2.393979,-2.094745,1.706787,2.200712,0.1770780
Wo_3,1.417819,1.167976,1.619066,0.4636685,0.1035803,0.3671289,0.2232538,-2.724542,1.408768,-3.430243,⋯,-7.358388,0.7925387,1.110468,0.9558195,0.01275816,2.396236,2.096720,-1.708396,-2.202787,-0.1772450


,Wo_1,Wo_2,Wo_3
,<dbl>,<dbl>,<dbl>
1,-0.329493108,-0.329493108,-0.329493108
2,-0.271430843,-0.271430843,-0.271430843
3,-0.376261658,-0.376261658,-0.376261658
4,-0.107753895,-0.107753895,-0.107753895
5,-0.024071463,-0.024071463,-0.024071463
6,-0.085318644,-0.085318644,-0.085318644
7,-0.051882914,-0.051882914,-0.051882914
8,0.633168004,0.633168004,0.633168004
9,-0.327389586,-0.327389586,-0.327389586
